# Map to analyse Global Land Cover!

In [1]:
import ee
from shapely.geometry import Polygon, box

from streamlit_map.geocoder import Geocoder
from streamlit_map.geodescriber import GeoDescriber
from streamlit_map.visualize import MapGEE, create_stacked_bar
from streamlit_map.processing import ZonalStatistics
from streamlit_map.data import GEEData

In [2]:
ee.Initialize()

In [3]:
MAX_ALLOWED_AREA_SIZE = 25.0

In [4]:
datasets = {}
for dataset in ['Global-Land-Cover', 'Koppen-Geiger-Climate']:
    datasets[dataset] = GEEData(dataset)
    
gee_data = datasets['Global-Land-Cover']

In [5]:
m = MapGEE()

m.add_gee_layer(
    image=gee_data.ee_image(),
    sld_interval=gee_data.sld_interval(),
    name=gee_data.dataset
)
m

Draw a rectangle on map to select and area.


MapGEE(center=[25.0, 55.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…